errors because they changed shooting stats season 2021-2922????

In [22]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [9]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [12]:
shooting.head()

For Manchester City                                                     \
                 Date   Time              Comp                Round  Day   
0          2021-08-07  17:15  Community Shield  FA Community Shield  Sat   
1          2021-08-15  16:30    Premier League          Matchweek 1  Sun   
2          2021-08-21  15:00    Premier League          Matchweek 2  Sat   
3          2021-08-28  12:30    Premier League          Matchweek 3  Sat   
4          2021-09-11  15:00    Premier League          Matchweek 4  Sat   

                                         ... Standard               Expected  \
     Venue Result GF GA        Opponent  ...     Dist   FK PK PKatt       xG   
0  Neutral      L  0  1  Leicester City  ...      NaN  NaN  0     0      NaN   
1     Away      L  0  1       Tottenham  ...     16.9  1.0  0     0      1.9   
2     Home      W  5  0    Norwich City  ...     17.3  1.0  0     0      2.7   
3     Home      W  5  0         Arsenal  ...     14.3  0.0  0     0      3.8   
4     Away      W  1  0  Leicester City  ...     14.0  0.0  0     0      2.9   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN     NaN  NaN     NaN        Match Report  
1  1.9    0.11 -1.9    -1.9        Match Report  
2  2.7    0.17  1.3     1.3        Match Report  
3  3.8    0.15  1.2     1.2        Match Report  
4  2.9    0.12 -1.9    -1.9        Match Report  

[5 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel()

In [14]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [15]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,...,4-3-3,Paul Tierney,Match Report,NaN,12,3,NaN,NaN,0,0
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,4-3-3,Anthony Taylor,Match Report,NaN,18,4,16.9,1.0,0,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,4-3-3,Graham Scott,Match Report,NaN,16,4,17.3,1.0,0,0
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,4-3-3,Martin Atkinson,Match Report,NaN,25,10,14.3,0.0,0,0
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,4-3-3,Paul Tierney,Match Report,NaN,25,8,14.0,0.0,0,0


In [16]:
years = list(range(2022, 2020, -1))
all_matches = []

In [17]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [33]:
import time

In [45]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

ValueError: No tables found

In [42]:
team_data = team_data[team_data["Comp"] == "Premier League"]

In [43]:
team_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
1,2019-08-09,20:00,Premier League,Matchweek 1,Fri,Home,W,4,1,Norwich City,...,Match Report,NaN,15,7,17.0,1.0,0,0,2022,Liverpool
3,2019-08-17,15:00,Premier League,Matchweek 2,Sat,Away,W,2,1,Southampton,...,Match Report,NaN,15,5,18.7,1.0,0,0,2022,Liverpool
4,2019-08-24,17:30,Premier League,Matchweek 3,Sat,Home,W,3,1,Arsenal,...,Match Report,NaN,24,4,18.5,0.0,1,1,2022,Liverpool
5,2019-08-31,17:30,Premier League,Matchweek 4,Sat,Away,W,3,0,Burnley,...,Match Report,NaN,15,6,20.6,0.0,0,0,2022,Liverpool
6,2019-09-14,12:30,Premier League,Matchweek 5,Sat,Home,W,3,1,Newcastle Utd,...,Match Report,NaN,20,8,13.7,0.0,0,0,2022,Liverpool
8,2019-09-22,16:30,Premier League,Matchweek 6,Sun,Away,W,2,1,Chelsea,...,Match Report,NaN,6,3,12.7,0.0,0,0,2022,Liverpool
10,2019-09-28,12:30,Premier League,Matchweek 7,Sat,Away,W,1,0,Sheffield Utd,...,Match Report,NaN,16,4,18.3,0.0,0,0,2022,Liverpool
12,2019-10-05,15:00,Premier League,Matchweek 8,Sat,Home,W,2,1,Leicester City,...,Match Report,NaN,17,7,15.1,1.0,1,1,2022,Liverpool
13,2019-10-20,16:30,Premier League,Matchweek 9,Sun,Away,D,1,1,Manchester Utd,...,Match Report,NaN,10,4,18.0,0.0,0,0,2022,Liverpool
15,2019-10-27,16:30,Premier League,Matchweek 10,Sun,Home,W,2,1,Tottenham,...,Match Report,NaN,20,12,15.2,1.0,1,1,2022,Liverpool


In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("matches.csv")